In [186]:
import pandas as pd
import numpy as np

df = pd.read_csv('landmarks_and_labels.csv')


len(df)

433

In [187]:
# Split label column
labels = df['label']
X = df.drop('label', axis=1)

# Convert labels to categorical
y = pd.get_dummies(labels)

# Split data into training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

y

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,True,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,True,False,False,False,False,False,False,False,False,False,False,False,False
4,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,False,False,False,False,False,False,False,False,False,False,False,False,False,True
429,False,False,False,False,False,False,False,False,False,False,False,False,True,False
430,False,False,False,False,False,False,False,False,False,False,False,False,False,True
431,False,False,False,False,False,False,False,False,False,False,False,False,True,False


In [188]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import numpy as np

# Define your grid search parameters
neurons_list = [128, 256, 512]  # Example: varying number of neurons
dropout_list = [0.2, 0.3, 0.4]  # Example: varying dropout rates
l1_list = [1e-5, 1e-6]
l2_list = [1e-4, 1e-3]

# Placeholder for best configuration
best_acc = -np.inf
best_config = None

for neurons in neurons_list:
    for dropout_rate in dropout_list:
        for l1 in l1_list:
            for l2 in l2_list:
                print(
                    f"Training model with {neurons} neurons, dropout {dropout_rate}, l1 {l1}, l2 {l2}"
                )

                # Define L1 and L2 regularizers dynamically
                regularizers = l1_l2(l1=l1, l2=l2)

                # Rebuild the model with the new parameters
                model = Sequential(
                    [
                        Dense(
                            neurons,
                            input_shape=(X_train.shape[1],),
                            kernel_regularizer=regularizers,
                        ),
                        BatchNormalization(),
                        Activation("swish"),
                        Dropout(dropout_rate),
                        Dense(
                            int(neurons / 2), kernel_regularizer=regularizers
                        ),  # Dynamically reduce neuron count for the second layer
                        BatchNormalization(),
                        Activation("swish"),
                        Dropout(dropout_rate),
                        Dense(y_train.shape[1], activation="softmax"),
                    ]
                )

                optimizer = Adam(learning_rate=0.001)
                model.compile(
                    optimizer=optimizer,
                    loss="categorical_crossentropy",
                    metrics=["accuracy"],
                )

                # Fit the model (consider using a smaller number of epochs for grid search)
                history = model.fit(
                    X_train, y_train, epochs=50, validation_split=0.1, verbose=0
                )  # Use verbose=0 for less output

                # Evaluate the model
                val_acc = np.max(
                    history.history["val_accuracy"]
                )  # Get the best validation accuracy

                # Update best configuration if current model is better
                if val_acc > best_acc:
                    best_acc = val_acc
                    best_config = (neurons, dropout_rate, l1, l2)
                    # Consider saving the model here if you want to keep the best model
                    model.save("best_model_grid_search.h5")

print(
    f"Best configuration: Neurons: {best_config[0]}, Dropout Rate: {best_config[1]}, l1: {best_config[2]}, l2: {best_config[3]} with Validation Accuracy: {best_acc}"
)

Training model with 128 neurons, dropout 0.2, l1 1e-05, l2 0.0001


c:\Users\aaaab\.conda\envs\tf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training model with 128 neurons, dropout 0.2, l1 1e-05, l2 0.001


c:\Users\aaaab\.conda\envs\tf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training model with 128 neurons, dropout 0.2, l1 1e-06, l2 0.0001


...

Training model with 128 neurons, dropout 0.3, l1 1e-05, l2 0.0001
Training model with 128 neurons, dropout 0.3, l1 1e-05, l2 0.001
Training model with 128 neurons, dropout 0.3, l1 1e-06, l2 0.0001
Training model with 128 neurons, dropout 0.3, l1 1e-06, l2 0.001
Training model with 128 neurons, dropout 0.4, l1 1e-05, l2 0.0001
Training model with 128 neurons, dropout 0.4, l1 1e-05, l2 0.001
Training model with 128 neurons, dropout 0.4, l1 1e-06, l2 0.0001
Training model with 128 neurons, dropout 0.4, l1 1e-06, l2 0.001
Training model with 256 neurons, dropout 0.2, l1 1e-05, l2 0.0001
Training model with 256 neurons, dropout 0.2, l1 1e-05, l2 0.001
Training model with 256 neurons, dropout 0.2, l1 1e-06, l2 0.0001
Training model with 256 neurons, dropout 0.2, l1 1e-06, l2 0.001
Training model with 256 neurons, dropout 0.3, l1 1e-05, l2 0.0001
Training model with 256 neurons, dropout 0.3, l1 1e-05, l2 0.001
Training model with 256 neurons, dropout 0.3, l1 1e-06, l2 0.0001
Training model wi

c:\Users\aaaab\.conda\envs\tf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training model with 512 neurons, dropout 0.2, l1 1e-06, l2 0.0001


c:\Users\aaaab\.conda\envs\tf\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Training model with 512 neurons, dropout 0.2, l1 1e-06, l2 0.001
Training model with 512 neurons, dropout 0.3, l1 1e-05, l2 0.0001
Training model with 512 neurons, dropout 0.3, l1 1e-05, l2 0.001
Training model with 512 neurons, dropout 0.3, l1 1e-06, l2 0.0001
Training model with 512 neurons, dropout 0.3, l1 1e-06, l2 0.001
Training model with 512 neurons, dropout 0.4, l1 1e-05, l2 0.0001
Training model with 512 neurons, dropout 0.4, l1 1e-05, l2 0.001
Training model with 512 neurons, dropout 0.4, l1 1e-06, l2 0.0001
Training model with 512 neurons, dropout 0.4, l1 1e-06, l2 0.001
Best configuration: Neurons: 512, Dropout Rate: 0.2, l1: 1e-06, l2: 0.0001 with Validation Accuracy: 0.8857142925262451


In [189]:
# model.evaluate(X_test, y_test)

In [190]:
# # Save model
# model.save('model.h5')